這個中文書籍分類器的目的是根據提供的書籍資訊（書名、書籍介紹或博客來連結），判斷書籍的適讀對象，包括小孩、青少年或成人。

首先，使用者可以選擇提供書名、書籍介紹或博客來連結來搜尋一本中文書籍的適讀對象。
根據使用者提供的資訊，程式會從博客來網站搜尋該書籍的介紹，或者直接使用使用者提供的書籍介紹。

接著使用ArticutAPI進行中文斷詞，將書籍資訊進行斷詞，將文本轉換成詞彙序列。
讀取已有的訓練資料，包括書籍資訊斷詞和適讀對象等欄位，並將新的書籍資訊加入其中。
使用CountVectorizer對斷詞後的文本進行特徵提取。
將文本轉換成Bag-of-Words模型，把每個詞彙視為一個特徵，並計算每個詞彙在文本中的出現頻率。

然後，使用train_test_split函數將提取到的特徵和相應的適讀對象劃分為80%的訓練集和20%的測試集。
在訓練集上，使用MultinomialNB分類器進行模型訓練。
最終的分類正確率為0.81。

最後，對於新的文本進行預測，並根據模型預測結果顯示該書籍的適讀對象。
會根據使用者的選擇，顯示書籍介紹、繼續搜尋其他書籍或結束程式，幫助讀者選擇適合的書籍。

資料來源：政府資料開放平臺 臺灣出版新書預告書訊 https://data.gov.tw/dataset/6730 

In [2]:
# 整理資料＋把資料分類
import pandas as pd

isbn_df = pd.read_csv('isbn.csv')
title_category_df = pd.concat([isbn_df["書名"], isbn_df["適讀對象"]], axis=1)  # ignore_index=False
# display(title_category_df)

# 把書名重複的書拿掉
isbn_unique_df = title_category_df.drop_duplicates(subset="書名", keep="first")
# isbn_unique_df.to_csv('isbn_unique_df.csv', index=False) # index=False: 不存列編號
# display(isbn_unique_df)

# 把資料分成三類：小孩、青少年、成人
# print(set(isbn_unique_df["適讀對象"])) # {'嬰幼兒(0-3歲)', '樂齡', '幼兒(3-6歲)', '青少年', '成人(學術)', '成人(一般)', '兒童(6-12歲)'}
for index, row in isbn_unique_df.iterrows():
    x = row["適讀對象"]
    if x == "嬰幼兒(0-3歲)" or x == "幼兒(3-6歲)" or x == "兒童(6-12歲)":
        isbn_unique_df.loc[index, "適讀對象"] = "小孩"
    elif x == "青少年":
        isbn_unique_df.loc[index, "適讀對象"] = "青少年"
    elif x == "成人(學術)" or x == "成人(一般)":
        isbn_unique_df.loc[index, "適讀對象"] = "成人"
        
isbn_unique_df.to_csv('isbn_unique_df.csv', index=False) 
display(isbn_unique_df)


/var/folders/v2/393_xfxs1_14q1sm_p473p780000gp/T/ipykernel_2422/3738023983.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isbn_unique_df.loc[index, "適讀對象"] = "成人"
/var/folders/v2/393_xfxs1_14q1sm_p473p780000gp/T/ipykernel_2422/3738023983.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isbn_unique_df.loc[index, "適讀對象"] = "小孩"
/var/folders/v2/393_xfxs1_14q1sm_p473p780000gp/T/ipykernel_2422/3738023983.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,書名,適讀對象
0,像佛洛伊德一樣反應與思考,成人
1,為美好的世界獻上祝福!Fantastic Days,成人
3,樹葉物語= 나뭇잎 수업: 사계절 나뭇잎 투쟁기,成人
4,"我隨意,你盡量",成人
6,"酒食聖經: 食物與酒、咖啡、茶、礦泉水的完美搭配,七十三位權威主廚與仕酒師的頂尖意見",成人
...,...,...
5814,以農為榮= Proud of agriculture,成人
5815,同一個太陽 韓湘寧H.N.HAN,成人
5816,無常形: 大同大學媒體設計學系第十二屆畢業專刊= Tatung University med...,成人
5817,"默默之路: 華麗登場,斑馬先生的奇幻試煉= La route du silence",成人


In [5]:
# 找博客來連結
import requests  # 爬蟲用 (http requests)
from bs4 import BeautifulSoup  # 爬蟲用 (html)
from urllib.parse import urlparse, parse_qs  # 爬蟲用 (parse url)
import pandas as pd  
import time

# 用Google找書本的博客來連結
def search_book_in_google(title):
    search_query = f"博客來 {title}"
    url = f"https://www.google.com/search?q={search_query}"

    # GET request
    response = requests.get(url)
    response.raise_for_status()

    # Parse the HTML response
    soup = BeautifulSoup(response.text, 'html.parser')

    link = soup.find('a', href=lambda href: href and 'books.com.tw' in href)
    if link:
        parsed_url = urlparse(link['href'])
        params = parse_qs(parsed_url.query)
        for key in params.keys():
            if 'q' in key:
                for value in params[key]:
                    if "https://www.books.com.tw/products/" in value:
                        # print(f"Finished {row['書名']}") ###
                        return value

    return None

df = pd.read_csv("isbn_unique_df.csv", encoding='utf-8')

df['博客來連結'] = ""

# 用書名Google找第一個博客來連結
for index, row in df.iterrows():
    title = row['書名']
    link = search_book_in_google(title)
    if link:
        df.at[index, '博客來連結'] = link
    else:
        # 如果找不到連結就刪除那列
        df = df.drop(index)
    time.sleep(2)  

# Reset index 
df = df.reset_index(drop=True)

output_file = 'isbn_unique_df_with_links.csv'
df.to_csv(output_file, index=False, encoding='utf-8')

display(df)
print(f"Updated DataFrame saved to {output_file}.")


,書名,適讀對象,博客來連結
0,像佛洛伊德一樣反應與思考,成人,https://www.books.com.tw/products/0010955509
1,為美好的世界獻上祝福!Fantastic Days,成人,https://www.books.com.tw/products/M010144445?l...
2,"我隨意,你盡量",成人,https://www.books.com.tw/products/0010951856
3,"酒食聖經: 食物與酒、咖啡、茶、礦泉水的完美搭配,七十三位權威主廚與仕酒師的頂尖意見",成人,https://www.books.com.tw/products/0010952725
4,InDesign 2021超強數位排版達人必備工作術,成人,https://www.books.com.tw/products/0010956276?l...
...,...,...,...
3830,澄懷館茗砂= The antique collections of Cheng Huai C...,成人,https://www.books.com.tw/products/F010585669?l...
3831,魔法攻擊造成的物理傷害= Attack damage caused by magic attack,成人,https://www.books.com.tw/products/F014906618
3832,見南山居文叢,成人,https://www.books.com.tw/products/0010374621
3833,以農為榮= Proud of agriculture,成人,https://www.books.com.tw/products/0010681270


Updated DataFrame saved to isbn_unique_df_with_links.csv.


In [ ]:
# 找書籍介紹
import pandas as pd
from selenium.webdriver.common.by import By  # 爬蟲用
from selenium import webdriver  # 爬蟲用
from webdriver_manager.chrome import ChromeDriverManager  # 爬蟲用
from selenium.webdriver.chrome.options import Options  # 爬蟲用
from selenium.common.exceptions import NoSuchElementException, WebDriverException  # 爬蟲用
import time

options = Options()
options.add_argument("--disable-notifications")
# options.add_argument("--headless")  # run Chrome in headless mode

service = webdriver.chrome.service.Service(ChromeDriverManager().install())
chrome = webdriver.Chrome(service=service, options=options)

df = pd.read_csv('book_information.csv', encoding='utf-8') # isbn_unique_df_with_links.csv

if '書籍資訊' not in df.columns:
    df['書籍資訊'] = ""

rows_to_drop = []

for index, row in df.iterrows():
    if pd.isnull(row['書籍資訊']):
        url = row['博客來連結']
        chrome.get(url)

        # 等頁面跑好
        time.sleep(5)

        try:
            content_div = chrome.find_element(By.CLASS_NAME, "content")
            information = content_div.text.strip()
        except (NoSuchElementException, WebDriverException):
            rows_to_drop.append(index)  
            continue

        df.at[index, '書籍資訊'] = information
        df.to_csv('book_information.csv', index=False, encoding='utf-8')

# 沒找到的那幾行刪掉
df.drop(rows_to_drop, inplace=True)

output_file = 'book_information.csv'
df.to_csv(output_file, index=False, encoding='utf-8')


In [44]:
display(df)

,書名,適讀對象,博客來連結,書籍資訊
0,像佛洛伊德一樣反應與思考,成人,https://www.books.com.tw/products/0010955509,看精神分析創始者佛洛伊德，大膽探索人類未知的心靈疆土，\n成為20世紀最具影響力的思想家！\...
1,為美好的世界獻上祝福!Fantastic Days,成人,https://www.books.com.tw/products/M010144445?l...,在異世界展開全新冒險生活\n\n描寫不幸遭遇意外的高中生佐藤和真，死後與女神阿克婭一起轉生到...
2,"我隨意,你盡量",成人,https://www.books.com.tw/products/0010951856,正港台文的淡水散步\n揮灑閒居小鎮的庶民生活風情畫，爬梳上淡水這個優勝美地的過去和現在，\n...
3,"酒食聖經: 食物與酒、咖啡、茶、礦泉水的完美搭配,七十三位權威主廚與仕酒師的頂尖意見",成人,https://www.books.com.tw/products/0010952725,最全面、最精采的食物與飲料搭配指南\n由食物與美酒共舞的絕妙滋味，沒有想像中那樣遙不可及\n...
4,InDesign 2021超強數位排版達人必備工作術,成人,https://www.books.com.tw/products/0010956276?l...,InDesign是Adobe公司所開發的版面編排軟體，透過本書的詳實介紹，可以讓初學者建立正...
...,...,...,...,...
3613,15-29歲青年勞⼯就業狀況調查報告. 111年,成人,https://www.books.com.tw/products/0010673154,蒐集青年勞工就業之基礎資料，掌握青年就業現況與問題，提供規劃青年就業政策參考。
3614,考試院施政編年錄. 中華民國一一一年,成人,https://www.books.com.tw/products/0010693394,考試院的施政計畫、考試院修正之條文及訂定之考試規則等、院會會議紀錄重點、討論事項、院內舉辦之...
3615,小圓找新家,小孩,https://www.books.com.tw/products/0010613756,有一天，小老鼠看到一個好大的蘋果，牠好想好想吃，但是家裡實在太小了，根本放不下這顆大蘋果。為...
3616,消費者保護研究. 第27輯,成人,https://www.books.com.tw/products/0010911838?l...,本書為本院消費者保護處譯介外國消費者保護法第27輯，內容為歐盟數位內容以及數位服務提供契約之...


In [69]:
# 利用articut斷詞
import json
import re
import pandas as pd
from time import sleep
from ArticutAPI import Articut

with open("FILE_PATH", 'r', encoding='utf8') as f:
    account = json.load(f)
username = account['username']
apikey = account['api_key']
articut = Articut(username, apikey)


df = pd.read_csv('book_information.csv', encoding='utf-8')

if '書籍資訊斷詞' not in df.columns:
    df['書籍資訊斷詞'] = ""
if 'articut.parse' not in df.columns:
    df['articut.parse'] = ""

# 如果之前有斷好的詞，就叫進來
segmented_df = pd.read_csv('book_segmented_information.csv', encoding='utf-8')
not_empty_indices = segmented_df['書籍資訊斷詞'].notnull()
df.loc[not_empty_indices, '書籍資訊斷詞'] = segmented_df.loc[not_empty_indices, '書籍資訊斷詞']
not_empty_indices_parse = segmented_df['articut.parse'].notnull()
df.loc[not_empty_indices_parse, 'articut.parse'] = segmented_df.loc[not_empty_indices_parse, 'articut.parse']


for index, text in enumerate(df['書籍資訊']):
    if pd.notnull(text):  # 判斷欄位是否為非空值
        text = text.replace('\n', '')
        resDICT = articut.parse(text)
        while not resDICT['status']:
            print(resDICT['msg'])
            sleep(2)
            resDICT = articut.parse(text)
        
        result_segmentation = resDICT['result_segmentation']
        result_segmentation = re.sub('/', ' ', result_segmentation)  # 把/換成空格（re.sub: 換好幾個，string.replace: 只能換一個），在這裡用哪種都可以
        df.at[index, '書籍資訊斷詞'] = result_segmentation
        df.at[index, 'articut.parse'] = resDICT  # 整個parse結果也存到df

        if result_segmentation:  # 判斷斷詞結果是否為非空值。老師說有時候不知道為什麼會產生空的資料。如果不是空的，才加進去
            df.at[index, '書籍資訊斷詞'] = result_segmentation
            # print(index, df.at[index, '書名']) 

        df.to_csv('book_segmented_information.csv', index=False, encoding='utf-8')

df.to_csv('book_segmented_information.csv', index=False) # index=False: 不存列編號


0 像佛洛伊德一樣反應與思考
1 為美好的世界獻上祝福!Fantastic Days
2 我隨意,你盡量
3 酒食聖經: 食物與酒、咖啡、茶、礦泉水的完美搭配,七十三位權威主廚與仕酒師的頂尖意見
4 InDesign 2021超強數位排版達人必備工作術
5 信託法
6 道家養生法: 道法自然,天人合一的養生觀= Taoist health method
7 消費者保護法
8 圖解無母數分析
9 經營之心
10 親愛的別怕,勇敢說yes: 關於生涯、職場、管理、家庭,鄒開蓮的20則人生提醒
11 講重點,高效能人士必備的說話基本功: 化繁為簡,輕鬆達標的精準溝通技巧
12 一流的貓系工作術
13 王子復仇記
14 教育原理
15 棒球驚嘆句. 2
16 跟大師學創造力. 8: 梵谷的藝術創造+21個藝術活動
17 鼓手名家技法
18 人生,不需要每一次都贏
19 圖解佛洛伊德與精神分析
20 Rozen Maiden0薔薇少女. 4
21 血界戰線Back 2 Back. 10
22 特調初戀 晴天咖啡館
23 MIX. 19
24 無職轉生: 到了異世界就拿出真本事, 失意的魔術師篇. 1
25 貴族轉生: 得天眷顧一出生就獲得最強力量. 2
26 間諜教室. 3
27 新人大叔冒險者,被最強隊伍操到死成無敵. 2
28 新網球王子. 33
29 達爾文遊戲. 23
30 嘆氣的亡靈想隱退. 2
31 暴食狂戰士 唯有我突破了所謂「等級」的概念. 2
32 樂園NOISE. 2
33 模擬後宮體驗. 5
34 模擬後宮體驗. 6
35 織田信奈的野望(全國版). 21
36 轉生成女性向遊戲只有毀滅END的壞人大小姐. 8
37 紫微斗數入門班
38 轉生成女性向遊戲只有毀滅END的壞人大小姐. 12
39 關於他的謊言與戀愛
40 跳出限制: 從想法到實際行動的指南-NLP
41 Asterigos失落迷城: 美術設定集
42 BLUE GIANT藍色巨星. 9
43 大唐封診錄. 第二部, 狩案司
44 重設界線: 善待自己,畫下恰到好處的界線,從此人生不再忍耐與疲累
45 隱形創傷: 成為大人的我們,該如何療癒看不見的童年傷痛?
46 融資性租賃契約之研究
47 ChatGPT領軍: DALL-E/Midjourney/D-ID/Synthesia: 邁向AI

In [4]:
# 只保留中文書
df = df[~df['書名'].str.contains(r'[a-zA-Z]')]
df.to_csv('book_segmented_information_Chinese_books.csv', index=False, encoding='utf-8')

In [14]:
# 全部斷詞使用MultinomialN做分類

# 向量化、拆成訓練集和測試集
from sklearn.feature_extraction.text import CountVectorizer # 文本先向量化。
from sklearn.model_selection import train_test_split  # 拆成訓練集和驗證集。
from sklearn.naive_bayes import MultinomialNB   # 給訓練集用。naive_bayes內除了MultinomialNB外還有很多種
from sklearn.metrics import accuracy_score # 給驗證集用。

df = pd.read_csv('book_segmented_information_Chinese_books.csv', encoding='utf-8')
df.dropna(subset=['書籍資訊斷詞'], inplace=True) # 使用 dropna() 函數刪除含有空值的行，以確保只保留非空值的行。

vec = CountVectorizer(token_pattern=r'(?u)\b\w+\b') # r'(?u)\b\w+\b': 一個字也要算
X = vec.fit_transform(df['書籍資訊斷詞']).toarray() # 所有文本的bow向量
y = df['適讀對象']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10) # 測試集：test_size=0.2 (0.訓練8:0.2測試, 0.9:0.1, ...)。random_state=10 隨意數字

nb = MultinomialNB()
nb.fit(X_train, y_train)  
y_predict = nb.predict(X_test)
print(f"驗證集的分類正確率：{round(accuracy_score(y_test, y_predict), 2)}") 
# xtrain要丟進去訓練的語料（教材），xtest等下要測驗的語料（考題），ytrain丟進去訓練的標籤（正確答案），ytest正確率標籤，y_predict解題結果


驗證集的分類正確率：0.81


In [7]:
# 可以直接看book_classification.py，較完整

# 讓使用者輸入想查詢的書籍
# 決定使用所有斷詞作分類
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urlparse, parse_qs

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from ArticutAPI import Articut
import pandas as pd
import requests
import time
import json
import re

with open("FILE_PATH", 'r', encoding='utf8') as f:
    account = json.load(f)
username = account['username']
apikey = account['api_key']
articut = Articut(username, apikey)

# 找博客來連結
def search_book_in_google(title):
    search_query = f"博客來 {title}"
    url = f"https://www.google.com/search?q={search_query}"

    # GET request
    response = requests.get(url)
    response.raise_for_status()
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到博客來的第一個連結
    link = soup.find('a', href=lambda href: href and 'books.com.tw' in href)
    if link:
        parsed_url = urlparse(link['href'])
        params = parse_qs(parsed_url.query)
        for key in params.keys():
            if 'q' in key:
                for value in params[key]:
                    if "https://www.books.com.tw/products/" in value:
                        # print(f"Finished {row['書名']}") ###
                        return value
    return None

# check if the text contains only Chinese
import unicodedata 
def is_chinese(text): 
    for char in text:
        # If the character is not in the 'Lo' (Letter, other) category, it is not Chinese
        if unicodedata.category(char) != 'Lo':
            return False
    return True # If all characters are in the 'Lo' category, the text is Chinese

print("您想搜尋哪本中文書？")
print("若您要提供書名，輸入1")
print("若您要提供書籍介紹，輸入2")
print("若您要提供博客來連結，輸入3")

information = None
title = None
url = None

while (True):
    option = input()

    if (option == "1" or option == "3"):
        if (option == "1"):
            while True:
                title = input("請輸入書名（中文）：")
                if is_chinese(title):
                    break
                else:
                    print("不好意思，您輸入不是中文，請重新輸入。")
                
            # 找博客來連結
            url = search_book_in_google(title)
            if not url:
                print("抱歉，我找不到此本書籍介紹，請您換一本。\n")
                print("您想搜尋哪本中文書？")
                print("若您要提供書名，輸入1")
                print("若您要提供書籍介紹，輸入2")
                print("若您要提供博客來連結，輸入3")
            # else:
                # print(url)

        elif (option == "3"):
            while True:
                url = input("請輸入博客來連結：")
                if "https://www.books.com.tw/products/" in url:
                    break
                else:
                    print("不好意思，您輸入的好像不是博客來連結，請重新輸入。")

        print("正在為您搜尋書籍介紹中，請稍候...")

        # 找書籍介紹
        options = Options()
        options.add_argument("--disable-notifications")
        options.add_argument("--headless")  # Add this option if you want to run Chrome in headless mode
        service = webdriver.chrome.service.Service(ChromeDriverManager().install())
        chrome = webdriver.Chrome(service=service, options=options)
        chrome.get(url)
        time.sleep(3)

        try:
            content_div = chrome.find_element(By.CLASS_NAME, "content")
            information = content_div.text.strip()
            chrome.quit()
            break
        except (NoSuchElementException, WebDriverException):
            chrome.quit()
            print("抱歉，我找不到此本書籍介紹，請您換一本")
            continue

    elif (option == "2"):
        while True:
            print("請輸入書籍介紹：")
            print("輸入完成後請按enter，接著按 Ctrl-D (mac) 或 Ctrl-Z (windows)")
            contents = []
            while True:
                try:
                    line = input()
                except EOFError:
                    break
                contents.append(line)
            information = str(contents)
            break

    else:
        print("請輸入1、2或3: (1: 提供書名, 2: 提供書籍介紹, 3: 提供博客來連結)")
        

print("正在為您分析中，請稍候...")

# 新書
newParse = articut.parse(information)
newSeg = newParse['result_segmentation']
newSeg = re.sub('/',' ', newSeg)

# 第一步，得把上面的句子斷詞，加到原來訓練的語料的pd.DataFrame中
# 讀入原來的評語資料，把上面新評語做斷詞後，存成newSegDF，
# 再使用pd.concat()把兩個DataFrame合併，axis = 0 表示垂直合併
df = pd.read_csv('book_segmented_information_Chinese_books.csv', encoding='utf-8')
df.dropna(subset=['書籍資訊斷詞'], inplace=True) # 使用 dropna() 函數刪除含有空值的行，以確保只保留非空值的行。
newBookDF = pd.DataFrame({"書籍資訊":[information], "博客來連結":url, "書籍資訊斷詞":newSeg, "適讀對象":["成人"], "articut.parse": str(newParse), "書名": title}) # 先把新的兩則和舊的df合併再一起，這樣bow才會有每個字，隨意設定label
evaDataDF = pd.concat([df, newBookDF], axis=0) 
# display(evaDataDF.tail())

# 第二步，把上面的concatData轉成BOW
vecArticut = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
newX = vecArticut.fit_transform(evaDataDF['書籍資訊斷詞']).toarray()
newY = evaDataDF['適讀對象']
X_train, X_test, y_train, y_test = train_test_split(newX[:-1], newY[:-1], test_size=0.2, random_state=10)  # newX[:-1]剛剛新加入的最後一行不拿去訓練，因為是最後才要預測的評語

# 第三步：訓練
nb = MultinomialNB()
nb.fit(X_train, y_train)

# 第四步：預測
# 因為nb.predict()帶一個2D array，
# 只有一句的話，就得再放入list中 -> [newX[-1]]
if option == "1" or option == "2":
    print(f"\n{title}的試讀對象是{nb.predict([newX[-1]])[0]}\n")

if option == "3":
    print(f"\n此本書的試讀對象是{nb.predict([newX[-1]])[0]}\n")

if option == "1" or option == "3":
    print(f"以下是書籍介紹:\n{information}")


您想搜尋哪本中文書？
若您要提供書名，輸入1
若您要提供書籍介紹，輸入2
若您要提供博客來連結，輸入3
正在為您搜尋書籍介紹中，請稍候...
正在為您分析中，請稍候...

我的爸爸的試讀對象是小孩

以下是書籍介紹:
獻給全天下默默守護兒女的父親
用繪本大聲念出對爸爸的愛，每天都是父親節！

繪本大師土田義晴經典名作
日本各大童書網站，每年父親節必選必推繪本！

爸爸總是忙，可是只要「我」往他窗邊一站，他就會說：「我們走吧！」讓我跳上他的自行車，去看櫻花，看田野，看小河……我會撿起一片片花瓣、三葉草、玫瑰花，還有好多樹果，放在我最心愛的手帕裡，然後來到小河邊，隨流水漂向大海……

有天，我的手帕掉了，我嚎淘大哭，因為那是我最珍愛的手帕，媽媽為我繡上小花的手帕……，媽媽留給我的唯一的紀念品。

我不理會爸爸的安慰，依舊任性的放聲大哭。

那天晚上，爸爸工作好晚好晚，一晚都沒睡……

隔天早上，我收到了一個意想不到的大驚喜！

我決定不再哭了，因為只要我一哭，爸爸比我還難過，而且，在這世界上，我最喜歡最喜歡爸爸了！

--------------------------------------------------------------------------

本書以溫柔細膩的筆觸，以真誠包容的文字，傾聽一個孩子講述自己經歷的離別和思念，和爸爸一起承擔生命中的驟逝無常，故事的最後，小女孩帶我們去看她的媽媽，「媽媽住在海邊。」一座被花草圍繞的墓碑，女兒和爸爸在海邊嬉戲，就像在媽媽的懷抱裡。

讀《我的爸爸》，讓孩子學會「勇敢」，這樣的勇敢不是希望孩子遇事臨危不亂，而是讓內心堅強，並看見或想起自己所擁有的──父親無時無刻全力以赴的愛。

名人推薦

吳鳳  金鐘獎主持人、作家
陳蕙慧 資深出版人
黃筱茵 童書翻譯評論工作者
劉旭恭 繪本作家
鄭宇庭 新手書店奶爸店長
盧方方 後青春繪本館主編
──爸爸愛最大推薦（按姓氏筆畫排列）


以下是最後沒用到的分類，正確率都是0.81

In [12]:
# 用實詞訓練
import json
import ast
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from ArticutAPI import Articut

with open("FILE_PATH", 'r', encoding='utf8') as f:
    account = json.load(f)
username = account['username']
apikey = account['api_key']
articut = Articut(username, apikey)

df = pd.read_csv('book_segmented_information_Chinese_books.csv', encoding='utf-8') ### book_segmented_information_test.csv
df.dropna(subset=['書籍資訊斷詞', 'articut.parse'], inplace=True) # 使用 dropna() 函數刪除含有空值的行，以確保只保留非空值的行。
# print(len(df))
# display(df['articut.parse'])

resDICT = {}
TokensDICT = {}
for index, parsed_result_str in enumerate(df['articut.parse']):
    parsed_result_dict = ast.literal_eval(parsed_result_str)
    resDICT[index] = parsed_result_dict
    # print(resDICT)
    TokensDICT.update({df.iloc[index]['書名']:resDICT[index]})  
# print(TokensDICT)

resContentTokens = []
for book in TokensDICT:
    # print(book)
    # print(TokensDICT[book])
    # print(articut.getContentWordLIST(TokensDICT[book]))
    contentWDs = articut.getContentWordLIST(TokensDICT[book])
    # print(contentWDs)
    contentWDs = sum(contentWDs, [])
    # print(contentWDs)
    contentTokens = []
    for t in contentWDs:
        contentTokens.append(t[-1])
    contentTokens = ' '.join(contentTokens)
    resContentTokens.append(contentTokens) 
# print(len(resContentTokens)) # 檢查數量是否正確

contentWDsDF = pd.DataFrame({'實詞':resContentTokens})
contentdf = pd.concat([df, contentWDsDF], axis=1)
contentdf.dropna(subset=['書籍資訊斷詞', 'articut.parse', '實詞'], inplace=True)
contentdf.to_csv('book_segmented_information_contentWord.csv', index=False, encoding='utf-8')
# display(contentdf)

contentWDs = contentdf['實詞'] 
vec = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
X = vec.fit_transform(contentWDs).toarray()
y = contentdf['適讀對象']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=10)
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
accuracy = round(accuracy_score(y_test, y_pred), 2)
print(f"用實詞分類的正確率：{accuracy}")


用實詞分類的正確率：0.81


In [13]:
# 動詞跟名詞來做分類，calculate the classification accuracy。

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score 

df = pd.read_csv('book_segmented_information_Chinese_books.csv', encoding='utf-8')
df.dropna(subset=['書籍資訊斷詞', 'articut.parse'], inplace=True) # 使用 dropna() 函數刪除含有空值的行，以確保只保留非空值的行。

resDICT = {}
TokensDICT = {}
for index, parsed_result_str in enumerate(df['articut.parse']):
    parsed_result_dict = ast.literal_eval(parsed_result_str)
    resDICT[index] = parsed_result_dict
    # print(resDICT)
    TokensDICT.update({df.iloc[index]['書名']:resDICT[index]})  
# print(TokensDICT)

NounVerbWords = []
for book in TokensDICT:
    nounWDs = articut.getNounStemLIST(TokensDICT[book])
    verbWDs = articut.getVerbStemLIST(TokensDICT[book])
    nounWDs = sum(nounWDs, [])
    verbWDs = sum(verbWDs, [])
    nounTokens = []
    verbTokens = []
    for t in nounWDs:
        nounTokens.append(t[-1])
    for t in verbWDs:
        verbTokens.append(t[-1])
    NounVerbWords.append(' '.join(nounTokens + verbTokens))
# print(len(NounVerbWords)) # 檢查數量是否正確

NounVerbWDsDF = pd.DataFrame({'名詞動詞':NounVerbWords})
nounVerbDF = pd.concat([df, NounVerbWDsDF], axis=1)
nounVerbDF.dropna(subset=['書籍資訊斷詞', 'articut.parse', '名詞動詞'], inplace=True)
nounVerbDF.to_csv('book_segmented_information_NounVerb.csv', index=False, encoding='utf-8')

y = df['適讀對象']
vec = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vec.fit_transform(NounVerbWords)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
accuracy = round(accuracy_score(y_test, y_pred), 2)
print(f"用動詞跟名詞分類的正確率：{accuracy}")
# print(nb.n_features_in_) # number of distinct words or tokens used in the vectorized input data

用動詞跟名詞分類的正確率：0.81
